<a href="https://colab.research.google.com/github/VoltAmpereWatt/homework/blob/master/project_PGO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install https://download.pytorch.org/whl/cu80/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

In [0]:
from google.colab import files,drive
uploaded  = files.upload()
# drive.mount('/content/drive')

Saving branch.csv to branch.csv
Saving bus.csv to bus.csv
Saving gen.csv to gen.csv


In [0]:
import io
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,LabelBinarizer, MinMaxScaler
from sklearn.model_selection import ShuffleSplit
import torch
import torch.nn as nn

In [0]:
branch = pd.read_csv('branch.csv',
                     names=['fbus',	'tbus',	'r',	'x',	'b',	'rateA',	'rateB',	
                              'rateC',	'ratio',	'angle',	'status',	'angmin',	'angmax'])

In [0]:
bus = pd.read_csv('bus.csv',names=['bus_number','type','P_demand','Q_demand',
                                   'shunt_conductance','shunt_susceptance','area',
                                   'V_magnitude','V_angle','baseKV','zone','Vmax','Vmin'])
bus.drop(['area','zone'],axis = 1)
label_binarizer = LabelBinarizer()
bus.fillna(value = 0,inplace=True)

In [0]:
gen = pd.read_csv('gen.csv',names = ['bus_number','P_gen','Q_gen','Qoutput_max',
                                     'Qoutput_min','Vg','total_base_MVA','machine_status',
                                     'Pmax','Pmin','Pc1','Pc2','Qc1min','Qc1max','Qc2min',
                                     'Qc2max','ramp_agc',
                                     'ramp_10','ramp_30','ramp_q','apf'])

gen = gen.drop(columns = ['Vg','total_base_MVA','machine_status','Pmax','Pmin',
                          'Pc1','Pc2','Qc1min','Qc1max','Qc2min','Qc2max','ramp_agc',
                          'ramp_10','ramp_30','ramp_q','apf'])

In [0]:
bus_number = np.array(bus['bus_number'])
gen_bus_number = np.array(gen['bus_number'])

Combining generator and bus datsets based on *bus_number* column.

In [0]:
bus = bus.merge(gen,how='left', on='bus_number')

In [0]:
voltage = pd.Series(bus['V_magnitude'])
angle = pd.Series(bus['V_angle'])
active_power = pd.Series(bus['P_demand'])
reactive_power = pd.Series(bus['Q_demand'])
active_gen_power = pd.Series(bus['P_gen'])
active_gen_power.fillna(value=0,inplace=True)
reactive_gen_power = pd.Series(bus['Q_gen'])
reactive_gen_power.fillna(value=0,inplace=True)

In [0]:
bus_dataset = pd.concat([voltage,angle,active_power,
                     reactive_power,active_gen_power,
                     reactive_gen_power],axis=1)
col_names = list(bus_dataset.columns)

In [0]:
min_max_scaler = MinMaxScaler()
dataset_scaled = pd.DataFrame(min_max_scaler.fit_transform(bus_dataset))
dataset_scaled.columns = col_names
noise = np.random.normal(0,0.2,size=np.shape(dataset_scaled))
noisy_dataset_scaled = dataset_scaled + noise

Converting ```dataset_scaled``` and ```noisy_dataset_scaled``` to PyTorch tensors

In [0]:
# dataset_scaled = np.array(dataset_scaled)
# noisy_dataset_scaled = np.array(noisy_dataset_scaled)

In [0]:
# strat_shuffle_split = ShuffleSplit(n_splits = 15, test_size = 0.35, random_state = 42)
# n_splits = strat_shuffle_split.get_n_splits(noisy_dataset_scaled, dataset_scaled)
# for train_index,test_index in strat_shuffle_split.split(noisy_dataset_scaled, dataset_scaled):
#   X_train, X_test = noisy_dataset_scaled[train_index], noisy_dataset_scaled[test_index]
#   y_train, y_test = dataset_scaled[train_index], dataset_scaled[test_index]

Building the network

In [0]:
class PowerNet(nn.Module):
  def __init__(self, input_size = 6, output_size = 6, hidden_layer_1 = 48, hidden_layer_2 = 24): 
    super(PowerNet,self).__init__()
    self.linear_1 = nn.Linear(input_size, hidden_layer_1)
    self.sigmoid = nn.Sigmoid()
    self.linear_2 = nn.Linear(hidden_layer_1,hidden_layer_2)
    self.output = nn.Linear(hidden_layer_2,output_size)
  def forward(self,x):
    x = x.view(-1,6)
    hidden_out_1 = self.linear_1(x)
    hidden_out_2 = self.linear_2(hidden_out_1)
    out = self.output(hidden_out_2)
    out = self.sigmoid(out)
    return out
  

In [0]:
power_net = PowerNet()
learning_rate = 0.1

In [0]:
loss_measure = nn.MSELoss()
optimizer = torch.optim.Adam(power_net.parameters(), lr  = learning_rate)

In [0]:
# batch_size = 100
# n_iters = 300
# num_epochs = n_iters / (len(X_train) / batch_size)
# num_epochs = int(num_epochs)

num_epochs = 5

In [0]:
power_net.train()

PowerNet(
  (linear_1): Linear(in_features=6, out_features=48, bias=True)
  (sigmoid): Sigmoid()
  (linear_2): Linear(in_features=48, out_features=24, bias=True)
  (output): Linear(in_features=24, out_features=6, bias=True)
)

In [0]:
SS = ShuffleSplit(n_splits = 15, test_size = 0.35, random_state = 42)
for train_index, test_index in SS.split(noisy_dataset_scaled):
  train_noisy_bus_data = noisy_dataset_scaled.loc[train_index]
  train_bus_data = dataset_scaled.loc[test_index] 

,V_magnitude,V_angle,P_demand,Q_demand,P_gen,Q_gen
545,0.342246,0.610024,0.490964,0.347463,0.281892,0.177079
2142,0.391888,0.064616,0.555383,0.391074,0.281892,0.177079
1583,0.402606,0.500530,0.491074,0.349501,0.641198,0.221115
1206,0.403734,0.537442,0.491074,0.349501,0.461293,0.241896
45,0.293169,0.521845,0.491074,0.349501,0.281892,0.177079
2668,0.401478,0.453900,0.494102,0.352150,0.281892,0.177079
1765,0.185254,0.487181,0.491074,0.349501,0.281892,0.177079
2718,0.368195,0.561474,0.540487,0.409211,0.281892,0.177079
2140,0.413888,0.661565,0.509399,0.367027,0.281892,0.177079
2130,0.446607,0.402553,0.565612,0.404239,0.281892,0.177079
